<a href="https://colab.research.google.com/github/aitanasc/Algoritmos_optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos__Aitana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Aitana Sabater Camps  <br>
Url:https://github.com/aitanasc/Algoritmos_optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos__Aitana.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1YwIGuPZTjc3ovAP4bQ8FaC4pujEqHm3y#scrollTo=hVbXYX-RfPWh <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

El **espacio de soluciones** está compuesto por:

- Tomas (30 en total): Cada escena de la película que debe ser grabada.

- Actores (10 en total): Cada actor participa en algunas tomas y cobra por día trabajado.

- Días de grabación: Queremos minimizar la cantidad de días en los que los actores deben asistir.

- Cada solución es una asignación de tomas a días, respetando la restricción de máximo 6 tomas por día.

La **función objetivo** busca minimizar el número total de días en los que los actores deben trabajar.

**Restricciones**
- Cada toma debe asignarse exactamente a un día.
- - No se pueden grabar más de 6 tomas por día.
Un actor trabaja un día si aparece en alguna de las tomas de ese día.

In [ ]:
#Respuesta


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

El problema pertenece a la clase NP-Hard, ya que se trata de una variante del problema de empaquetamiento y asignación.
Si intentáramos evaluar todas las posibles combinaciones de tomas en días, tendríamos un espacio de soluciones exponencial.

Complejidad exacta: 𝑂(𝐷^𝑇) donde 𝐷 es el número de días y 𝑇 las tomas.

Al ser un problema NP-Hard, con restricciones combinatorias, no se puede resolver eficientemente para casos grandes con algoritmos exactos.

Opciones de resolución:

- Exacta: Ramificación y poda con CPLEX / OR-Tools.
- Heurística: Algoritmo voraz greedy.
- Metaheurísticas: Recocido simulado, algoritmo genético, GRASP.



#Diseño
- ¿Que técnica utilizo? ¿Por qué?

El código implementa un algoritmo heurístico greedy para minimizar los costos en la planificación de sesiones de doblaje. Este algoritmo:

- Ordena las tomas y las asigna a los días agrupando actores comunes.
- Llena los días con hasta 6 tomas, minimizando la cantidad de actores nuevos en cada sesión.
- Calcula el costo total en términos de actores que deben asistir cada día.

In [ ]:
#Respuesta

In [ ]:
import numpy as np
import pandas as pd

# Matriz de tomas (filas) x actores (columnas), 1 indica que el actor participa en la toma
actores_tomas = np.array([
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
])

num_tomas, num_actores = actores_tomas.shape
tomas = list(range(1, num_tomas + 1))  # Tomas empiezan en 1
actores_por_toma = {tomas[i]: set(np.where(actores_tomas[i] == 1)[0] + 1) for i in range(num_tomas)}  # Actores empiezan en 1

def heuristica_greedy(tomas, actores_por_toma, max_tomas_dia=6):
    tomas_pendientes = list(tomas)
    plan = []

    while tomas_pendientes:
        dia_actual = []
        actores_en_dia = set()

        # Seleccionar hasta 6 tomas priorizando aquellas que reutilizan actores del mismo día
        while len(dia_actual) < max_tomas_dia and tomas_pendientes:
            mejor_toma = None
            mejor_puntaje = float('-inf')

            for toma in tomas_pendientes:
                actores_nuevos = actores_por_toma[toma] - actores_en_dia
                puntaje = len(actores_por_toma[toma] & actores_en_dia) - len(actores_nuevos)

                if puntaje > mejor_puntaje:
                    mejor_toma = toma
                    mejor_puntaje = puntaje

            if mejor_toma is not None:
                dia_actual.append(mejor_toma)
                actores_en_dia.update(actores_por_toma[mejor_toma])
                tomas_pendientes.remove(mejor_toma)

        # Si aún hay espacio, agregar tomas hasta completar 6
        while len(dia_actual) < max_tomas_dia and tomas_pendientes:
            dia_actual.append(tomas_pendientes.pop(0))

        plan.append(dia_actual)

    return plan

# Aplicar la heurística y mostrar la planificación
plan_greedy = heuristica_greedy(tomas, actores_por_toma)

# Calcular los actores que trabajan cada día y el costo total
actores_por_dia = []
costo_total = 0

for dia in plan_greedy:
    actores_en_dia = set()
    for toma in dia:
        actores_en_dia.update(actores_por_toma[toma])
    actores_por_dia.append(sorted(list(actores_en_dia)))
    costo_total += len(actores_en_dia)

# Crear DataFrame con la planificación detallada
df_planificacion = pd.DataFrame({
    "Día": range(1, len(plan_greedy) + 1),
    "Tomas": plan_greedy,
    "Actores": actores_por_dia,
    "Costo (Actores en el Día)": [len(actores) for actores in actores_por_dia]
})

# Mostrar resultados
print(df_planificacion)
print(f" Costo total estimado: {costo_total}")

   Día                     Tomas                   Actores  \
0    1      [16, 25, 9, 6, 7, 1]       [1, 2, 3, 4, 5, 10]   
1    2  [17, 19, 23, 14, 18, 24]                 [1, 3, 6]   
2    3    [21, 5, 8, 12, 22, 11]     [1, 2, 3, 4, 5, 6, 8]   
3    4   [27, 2, 20, 13, 26, 28]           [1, 3, 4, 5, 9]   
4    5    [30, 15, 4, 3, 29, 10]  [1, 2, 4, 5, 6, 7, 8, 9]   

   Costo (Actores en el Día)  
0                          6  
1                          3  
2                          7  
3                          5  
4                          8  
 Costo total estimado: 29


#Conclusiones


Este trabajo ilustra el uso de técnicas heurísticas para resolver problemas de optimización combinatoria. Se ha conseguido una planificación válida que minimiza el coste total.

Para validar que no existe otra solución mejor, se ha formulado y resuelto también un modelo MILP con OR-Tools. El coste obtenido con el algoritmo heurístico greedy (29 actores-día) coincide con el coste mínimo alcanzado por el modelo MILP, confirmando que, en este caso, la solución greedy es óptima.

Este enfoque podría complementarse con un algoritmo genético o GRASP para mejorar la calidad de la solución en casos más grandes o menos estructurados.